In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm
import os

In [8]:
os.chdir("D://meridianthe4//PML//Cases//Glass_Identification")

In [9]:
glass = pd.read_csv("Glass.csv")
X, y = glass.drop('Type', axis=1), glass['Type']
le = LabelEncoder()
y = le.fit_transform(y)
lr = LogisticRegression(max_iter=5000)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
results = cross_val_score(lr, X, y, cv=kfold, scoring='accuracy')

In [10]:
np.mean(results)

np.float64(0.6118493909191584)

In [17]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [23]:
results = cross_val_score(lr, X, y, cv=kfold, scoring='f1_macro')
np.mean(results)

np.float64(0.5693910680822445)

In [24]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = cross_val_score(lr, X, y, cv=skfold, scoring='f1_macro')
np.mean(results)

np.float64(0.5735221965135759)

In [18]:
solvers = ['newton-cg', 'lbfgs', 'newton-cholesky', 'sag', 'saga']
Cs = np.linspace(0.001, 15, 20)
penalties = ['l2', None]
scores = []
for solver in tqdm(solvers):
    for C in Cs:
        for penalty in penalties:
            if penalty is None:
                lr = LogisticRegression(max_iter=50000, solver=solver, penalty=penalty)
                result = cross_val_score(lr, X, y, cv=kfold, scoring='f1_macro')
                scores.append([solver, C, penalty, np.mean(result)])
            else:
                lr = LogisticRegression(max_iter=50000, solver=solver, C=C, penalty=penalty)
                result = cross_val_score(lr, X, y, cv=kfold, scoring='f1_macro')
                scores.append([solver, C, penalty, np.mean(result)])
df_scores = pd.DataFrame(scores, columns=['solver', 'C', 'penalty', 'score'])
df_scores.sort_values(by='score', ascending=False)

100%|██████████| 5/5 [01:00<00:00, 12.01s/it]


,solver,C,penalty,score
4,newton-cg,1.579842,l2,0.585337
44,lbfgs,1.579842,l2,0.585337
124,sag,1.579842,l2,0.585337
84,newton-cholesky,1.579842,l2,0.585337
121,sag,0.001000,None,0.581631
...,...,...,...,...
0,newton-cg,0.001000,l2,0.108327
40,lbfgs,0.001000,l2,0.108327
80,newton-cholesky,0.001000,l2,0.108327
120,sag,0.001000,l2,0.108327


In [20]:
depths = [None, 3, 4, 5, 6, 7]
min_samples = [2, 10, 0.025, 0.01, 0.05, 0.1]
min_leaf = [1, 10, 0.025, 0.01, 0.05, 0.1]
scores = []
for depth in tqdm(depths):
    for min_sample in min_samples:
        for leaf in min_leaf:
            dt = DecisionTreeClassifier(max_depth=depth, min_samples_split=min_sample, min_samples_leaf=leaf)
            result = cross_val_score(dt, X, y, cv=kfold, scoring='f1_macro')
            scores.append([depth, min_sample, leaf, np.mean(result)])

100%|██████████| 6/6 [00:03<00:00,  1.83it/s]


In [21]:
df_scores = pd.DataFrame(scores, columns=['solver', 'C', 'penalty', 'score'])
df_scores.sort_values(by='score', ascending=False)

,solver,C,penalty,score
183,7.0,2.000,0.01,0.708861
186,7.0,10.000,1.00,0.707932
24,NaN,0.050,1.00,0.699213
117,5.0,10.000,0.01,0.697816
165,6.0,0.010,0.01,0.693263
...,...,...,...,...
77,4.0,2.000,0.10,0.451427
101,4.0,0.050,0.10,0.451427
95,4.0,0.010,0.10,0.451427
89,4.0,0.025,0.10,0.451427


### Stratified K-Fold

In [29]:
solvers = ['newton-cg', 'lbfgs', 'sag', 'saga']
Cs = np.linspace(0.001, 15, 20)
penalties = ['l2', None]
scores = []
for solver in tqdm(solvers):
    for C in Cs:
        for penalty in penalties:
            if penalty is None:
                lr = LogisticRegression(max_iter=50000, solver=solver, penalty=penalty)
                result = cross_val_score(lr, X, y, cv=skfold, scoring='f1_macro')
                scores.append([solver, C, penalty, np.mean(result)])
            else:
                lr = LogisticRegression(max_iter=50000, solver=solver, C=C, penalty=penalty)
                result = cross_val_score(lr, X, y, cv=skfold, scoring='f1_macro')
                scores.append([solver, C, penalty, np.mean(result)])
df_scores = pd.DataFrame(scores, columns=['solver', 'C', 'penalty', 'score'])
df_scores.sort_values(by='score', ascending=False)

100%|██████████| 4/4 [01:02<00:00, 15.61s/it]


,solver,C,penalty,score
78,lbfgs,15.000000,l2,0.604381
38,newton-cg,15.000000,l2,0.604381
118,sag,15.000000,l2,0.583526
36,newton-cg,14.210579,l2,0.583526
158,saga,15.000000,l2,0.583526
...,...,...,...,...
84,sag,1.579842,l2,0.536016
0,newton-cg,0.001000,l2,0.096895
40,lbfgs,0.001000,l2,0.096895
80,sag,0.001000,l2,0.096895


In [28]:
depths = [None, 3, 4, 5, 6, 7]
min_samples = [2, 10, 0.025, 0.01, 0.05, 0.1]
min_leaf = [1, 10, 0.025, 0.01, 0.05, 0.1]
scores = []
for depth in tqdm(depths):
    for min_sample in min_samples:
        for leaf in min_leaf:
            dt = DecisionTreeClassifier(max_depth=depth, min_samples_split=min_sample, min_samples_leaf=leaf)
            result = cross_val_score(dt, X, y, cv=skfold, scoring='f1_macro')
            scores.append([depth, min_sample, leaf, np.mean(result)])
df_scores = pd.DataFrame(scores, columns=['solver', 'C', 'penalty', 'score'])
df_scores.sort_values(by='score', ascending=False)

100%|██████████| 6/6 [00:03<00:00,  1.86it/s]


,solver,C,penalty,score
3,NaN,2.000,0.01,0.671623
15,NaN,0.025,0.01,0.667218
9,NaN,10.000,0.01,0.665337
186,7.0,10.000,1.00,0.663722
189,7.0,10.000,0.01,0.662408
...,...,...,...,...
59,3.0,0.010,0.10,0.459169
47,3.0,10.000,0.10,0.459169
41,3.0,2.000,0.10,0.459169
71,3.0,0.100,0.10,0.459169
